In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import cv2
import time, sys
from IPython.display import clear_output, display
from time import sleep
import random
from tabulate import tabulate

In [ ]:
%run -i ./Classes.ipynb

In [ ]:
# Load
ori_img = cv2.imread('./assets/peppers.png') #BGR
ori_logo = cv2.imread('./assets/shuriken.png', 0)

image_size = 256
logo_size = 20
q_step = 0.5
tf_step = 30


ImageUtil.show1(ori_img, 'Image', 4)
ImageUtil.show1(ori_logo, 'Logo', 4)

In [ ]:
def get_attacked_version(image, image_size_):
    
    a_fa = Attacker.filter_average(image, 3)
    a_nsp = Attacker.noise_salt_pepper(image, 0.01)
    
    resize_size = int(0.75 * image_size_)
    if(resize_size % 2 == 1): resize_size = resize_size + 1
    a_res = Attacker.resize(image, resize_size)
    a_rot = Attacker.rotate(image, -45)
    a_cro = Attacker.crop(image, 0.03, 0.03, 0.03, 0.03)
    a_jpg = Attacker.jpeg_compress(image, 75)
    
    return [
        Image(np.copy(image), image_size_),
        Image(a_fa, image_size_),
        Image(a_nsp, image_size_),
        Image(a_res, resize_size),
        Image(a_rot, image_size_),
        Image(a_cro, image_size_),
        Image(a_jpg, image_size_),
    ]

# Test Quantization Step

In [ ]:
test_d_results = []
test_d_images = []
test_d_logos = []

for dd in [0.2, 0.4, 0.6, 0.8, 1]:
# for dd in [0.4, 0.6]:
    
    # Parameter setup
    n = image_size
    g = logo_size
    d = dd
    
    # Initialize
    test_image = Image(ori_img, n)
    test_logo = Logo(ori_logo, g)

    # Create Watermarker
    emb_riw = RIW(n, g, 2 * g, d, tf_step)

    # Embed
    img_y_watermarked = emb_riw.embed(test_image.get_y(), test_logo.logo)
    watermarked_image = Image.copy(test_image)
    watermarked_image.set_y(img_y_watermarked)

    # Calculate PSNR
    psnr = ImageUtil.psnr(test_image.image, watermarked_image.image)

    # Attack
    attacked_images = get_attacked_version(watermarked_image.image, n)
    
    temp_images = []
    temp_logos = []
    temp_results = [n, g, d, psnr]
    
    # Process All attacked images
    for attacked_image in attacked_images:
        
        ext_riw = RIW(attacked_image.image_size, g, 2 * g, d, tf_step)
        
        # Extract
        logo_extracted = ext_riw.extract(attacked_image.get_y())

        # Calculate BER
        ber = ImageUtil.ber(test_logo.logo, logo_extracted)
        
        temp_images.append(attacked_image.image)
        temp_logos.append(logo_extracted)
        temp_results.append(ber)
        
    test_d_images.append(temp_images)
    test_d_logos.append(temp_logos)
    test_d_results.append(temp_results)

In [ ]:
print(tabulate(test_d_results, headers=['Image Size', 'Logo Size', 'Q Step', 'PSNR', 'BER', 'BER', 'BER', 'BER', 'BER', 'BER', 'BER']))

In [ ]:
ImageUtil.show(test_d_images, None, 5)

In [ ]:
ImageUtil.show(test_d_logos, None, 5)

# Test Logo Size

In [ ]:
test_g_results = []
test_g_images = []
test_g_logos = []

for gg in [12, 20, 28, 36]:
# for gg in [12, 20]:
    
    # Parameter setup
    n = image_size
    g = gg
    d = q_step
    
    # Initialize
    test_image = Image(ori_img, n)
    test_logo = Logo(ori_logo, g)

    # Create Watermarker
    emb_riw = RIW(n, g, 2 * g, d, tf_step)

    # Embed
    img_y_watermarked = emb_riw.embed(test_image.get_y(), test_logo.logo)
    watermarked_image = Image.copy(test_image)
    watermarked_image.set_y(img_y_watermarked)

    # Calculate PSNR
    psnr = ImageUtil.psnr(test_image.image, watermarked_image.image)

    # Attack
    attacked_images = get_attacked_version(watermarked_image.image, n)
    
    temp_images = []
    temp_logos = []
    temp_results = [n, g, d, psnr]
    
    # Process All attacked images
    for attacked_image in attacked_images:
        
        ext_riw = RIW(attacked_image.image_size, g, 2 * g, d, tf_step)
        
        # Extract
        logo_extracted = ext_riw.extract(attacked_image.get_y())

        # Calculate BER
        ber = ImageUtil.ber(test_logo.logo, logo_extracted)
        
        temp_images.append(attacked_image.image)
        temp_logos.append(logo_extracted)
        temp_results.append(ber)
        
    test_g_images.append(temp_images)
    test_g_logos.append(temp_logos)
    test_g_results.append(temp_results)

In [ ]:
print(tabulate(test_g_results, headers=['Image Size', 'Logo Size', 'Q Step', 'PSNR', 'BER', 'BER', 'BER', 'BER', 'BER', 'BER', 'BER']))

In [ ]:
ImageUtil.show(test_g_images, None, 5)

In [ ]:
ImageUtil.show(test_g_logos, None, 5)

# Test Image Size

In [ ]:
test_n_results = []
test_n_images = []
test_n_logos = []

for nn in [100, 200, 300, 400]:
# for nn in [100, 200]:
    
    # Parameter setup
    n = nn
    g = logo_size
    d = q_step
    
    # Initialize
    test_image = Image(ori_img, n)
    test_logo = Logo(ori_logo, g)

    # Create Watermarker
    emb_riw = RIW(n, g, 2 * g, d, 50)

    # Embed
    img_y_watermarked = emb_riw.embed(test_image.get_y(), test_logo.logo)
    watermarked_image = Image.copy(test_image)
    watermarked_image.set_y(img_y_watermarked)

    # Calculate PSNR
    psnr = ImageUtil.psnr(test_image.image, watermarked_image.image)

    # Attack
    attacked_images = get_attacked_version(watermarked_image.image, n)
    
    temp_images = []
    temp_logos = []
    temp_results = [n, g, d, psnr]
    
    # Process All attacked images
    for attacked_image in attacked_images:
        
        ext_riw = RIW(attacked_image.image_size, g, 2 * g, d, 50)
        
        # Extract
        logo_extracted = ext_riw.extract(attacked_image.get_y())

        # Calculate BER
        ber = ImageUtil.ber(test_logo.logo, logo_extracted)
        
        temp_images.append(attacked_image.image)
        temp_logos.append(logo_extracted)
        temp_results.append(ber)
        
    test_n_images.append(temp_images)
    test_n_logos.append(temp_logos)
    test_n_results.append(temp_results)

In [ ]:
print(tabulate(test_n_results, headers=['Image Size', 'Logo Size', 'Q Step', 'PSNR', 'BER', 'BER', 'BER', 'BER', 'BER', 'BER', 'BER']))

In [ ]:
ImageUtil.show(test_n_images, None, 5)

In [ ]:
ImageUtil.show(test_n_logos, None, 5)